In [1]:
import string
import numpy as np
import re

In [2]:
# reading text

with open("/content/1661-0.txt","r",encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

580619

In [4]:
# dividing sentence
sen = text.split('.')

In [5]:
# take only valid sentences
sen = [i for i in sen if len(i) > 10]

In [6]:
len(sen)

6107

In [7]:
# remove escape sequence
sen = [i.replace('\n'," ") for i in sen]

In [8]:
sen = [i.replace("—"," ") for i in sen]

In [9]:
puncs = string.punctuation

trans = str.maketrans("","",puncs)
sen = [i.translate(trans) for i in sen]

In [10]:
pattern = r'[“”‘’—…]'

sen = [re.sub(pattern,"",i) for i in sen]

In [11]:
sen = [i.strip() for i in sen]

In [12]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [13]:
tokenizer = Tokenizer()

In [14]:
tokenizer.fit_on_texts(sen)

In [15]:
word_index = tokenizer.word_index

index_word = {j:i for i,j in word_index.items()}

In [16]:
vocab = len(tokenizer.word_index) + 1
vocab

8532

In [17]:
maxlen = max([len(i.split()) for i in sen])
maxlen

101

In [18]:
input_seq = []

for i in sen:
    tok = tokenizer.texts_to_sequences([i])[0]

    for j in range(1,len(tok)):
        seq = tok[:j+1]
        input_seq.append(seq)

In [19]:
pad_seq = pad_sequences(input_seq,maxlen=maxlen,padding='pre')

In [20]:
x = pad_seq[:,:-1]
y = pad_seq[:,-1]

In [21]:
y = to_categorical(y,num_classes=vocab)

In [22]:
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense

In [23]:
model = Sequential()

model.add(Embedding(vocab,100,input_length=maxlen-1))
model.add(LSTM(150,return_sequences=True))
model.add(LSTM(150))
model.add(Dense(vocab,activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          853200    
                                                                 
 lstm (LSTM)                 (None, 100, 150)          150600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 8532)              1288332   
                                                                 
Total params: 2472732 (9.43 MB)
Trainable params: 2472732 (9.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [26]:
h = model.fit(x,y,epochs=10)

Epoch 1/10
3163/3163 [==============================] - 83s 24ms/step - loss: 6.4149 - accuracy: 0.0720
Epoch 2/10
3163/3163 [==============================] - 48s 15ms/step - loss: 5.7231 - accuracy: 0.1096
Epoch 3/10
3163/3163 [==============================] - 46s 15ms/step - loss: 5.3745 - accuracy: 0.1311
Epoch 4/10
3163/3163 [==============================] - 46s 14ms/step - loss: 5.1206 - accuracy: 0.1430
Epoch 5/10
3163/3163 [==============================] - 49s 15ms/step - loss: 4.9122 - accuracy: 0.1543
Epoch 6/10
3163/3163 [==============================] - 52s 16ms/step - loss: 4.7284 - accuracy: 0.1624
Epoch 7/10
3163/3163 [==============================] - 50s 16ms/step - loss: 4.5558 - accuracy: 0.1693
Epoch 8/10
3163/3163 [==============================] - 44s 14ms/step - loss: 4.3918 - accuracy: 0.1754
Epoch 9/10
3163/3163 [==============================] - 46s 15ms/step - loss: 4.2365 - accuracy: 0.1840
Epoch 10/10
3163/3163 [==============================] - 43s 14m

In [27]:
h2 = model.fit(x,y,epochs=100,initial_epoch=10)

Epoch 11/100
3163/3163 [==============================] - 45s 14ms/step - loss: 3.9532 - accuracy: 0.2029
Epoch 12/100
3163/3163 [==============================] - 42s 13ms/step - loss: 3.8263 - accuracy: 0.2158
Epoch 13/100
3163/3163 [==============================] - 44s 14ms/step - loss: 3.7096 - accuracy: 0.2277
Epoch 14/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.5996 - accuracy: 0.2414
Epoch 15/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.4971 - accuracy: 0.2552
Epoch 16/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.4011 - accuracy: 0.2696
Epoch 17/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.3102 - accuracy: 0.2817
Epoch 18/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.2261 - accuracy: 0.2949
Epoch 19/100
3163/3163 [==============================] - 43s 14ms/step - loss: 3.1440 - accuracy: 0.3069
Epoch 20/100
3163/3163 [======================

In [28]:
model.save(r'model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
text = "My marriage"

for i in range(10):
    token_text = tokenizer.texts_to_sequences([text])[0]
    padded_token_text = pad_sequences([token_text], maxlen=maxlen-1, padding='pre')

    pos = np.argmax(model.predict(padded_token_text))

    text += " " + index_word[pos]
    print(text)

1/1 [==============================] - 0s 20ms/step
My marriage had
1/1 [==============================] - 0s 19ms/step
My marriage had drifted
1/1 [==============================] - 0s 19ms/step
My marriage had drifted us
1/1 [==============================] - 0s 18ms/step
My marriage had drifted us away
1/1 [==============================] - 0s 21ms/step
My marriage had drifted us away from
1/1 [==============================] - 0s 20ms/step
My marriage had drifted us away from each
1/1 [==============================] - 0s 20ms/step
My marriage had drifted us away from each other
1/1 [==============================] - 0s 19ms/step
My marriage had drifted us away from each other ways
1/1 [==============================] - 0s 20ms/step
My marriage had drifted us away from each other ways smoking
1/1 [==============================] - 0s 19ms/step
My marriage had drifted us away from each other ways smoking and
